# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Recopilación de datos** </center>

---

**Proyecto Final**

**Fecha**: 13 mayo 2025

**Nombre del Equipo**: Arriba Linux

**Integrantes del Equipo**: Tirzah Peniche Barba / Ana Cristina Luna Arellano / Juan Pedro Bihouet

**Profesor**: Dr. Pablo Camarillo Ramirez

In [8]:
import findspark
findspark.init()

### Creacion de la conexión con el cluster de spark


Se crea una sesión de Spark con el paquete de Kafka incluido.

In [9]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Arriba-Linux-Proyecto-Final") \
    .master("spark://ac7f0d7e8e91:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

spark.conf.set("spark.sql.shuffle.partitions", "5")

### Crear el stream desde Kafka

Se hace la conexión a Kafka para escuchar los dos topicos. Kafka entrega los datos en formato binario, por lo que se debe convertirlos. Se crearon los tópicos directamente en la consola de kafka para ahí subscribirse a la lectura de datos.

In [10]:
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "78a305ddc318:9093") \
                .option("subscribePattern", ".*_topic") \
                .option("startingOffsets", "latest") \
                .load()

kafka_lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



### Transformar los datos a columnas de strings

Se usa .withColumn para castear el string y luego descomponerlo para obtener colúmnas útiles. Ya que todo se está usando en strings, se tomó la decisión de usar cast("int") en el único dato diferente para simplificar el proceso.

In [11]:
from pyspark.sql.functions import split, col

kafka_df = kafka_lines.withColumn("value_str", kafka_lines.value.cast("string"))

social_df = kafka_df.select(
    split(col("value_str"), " \\| ").alias("fields")
).select(
    col("fields")[0].alias("timestamp"),
    col("fields")[1].alias("platform"),
    col("fields")[2].alias("user"),
    col("fields")[3].alias("text"),
    col("fields")[4].cast("int").alias("likes")
)


### Escribir el stream en formato Parquet (Data Lake)

Se guarda el resultado del stream como archivos Parquet en un directorio.

In [12]:
query = social_df.writeStream \
    .outputMode("append") \
    .trigger(processingTime='5 seconds') \
    .format("parquet") \
    .option("path", "/home/jovyan/notebooks/datalake/social_logs/") \
    .option("checkpointLocation", "/home/jovyan/notebooks/datalake/_checkpoints/") \
    .start()

query.awaitTermination(200)

25/05/13 02:57:50 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/13 02:57:50 WARN StreamingQueryManager: Stopping existing streaming query [id=736cf695-3fca-4dd5-bfa1-271f4661eafd, runId=c7bc72b0-c477-49ae-a193-05f8ae8cda5a], as a new run is being started.
25/05/13 02:57:58 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/05/13 02:57:58 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 7517 milliseconds


False

In [13]:
df = spark.read.parquet("/home/jovyan/notebooks/datalake/social_logs/")
df.show(10, truncate=False)


+-------------------+---------+-------+-------------------------+-----+
|timestamp          |platform |user   |text                     |likes|
+-------------------+---------+-------+-------------------------+-----+
|2025-05-13 02:23:21|instagram|user_8 |My heart is full         |1939 |
|2025-05-13 02:23:21|instagram|user_25|Look at this outfit!     |2268 |
|2025-05-13 02:23:21|instagram|user_37|Throwback to last summer |2110 |
|2025-05-13 02:23:21|instagram|user_43|So in love with this meal|2920 |
|2025-05-13 02:23:21|instagram|user_10|Morning vibes            |80   |
|2025-05-13 02:23:21|instagram|user_39|Besties forever          |346  |
|2025-05-13 02:23:21|instagram|user_50|This place is magical    |1301 |
|2025-05-13 02:23:21|instagram|user_21|Beach days are the best  |1600 |
|2025-05-13 02:23:21|instagram|user_13|My heart is full         |755  |
|2025-05-13 02:23:21|instagram|user_24|Throwback to last summer |4326 |
+-------------------+---------+-------+-------------------------

In [ ]:
sc.stop()

25/05/13 03:23:50 ERROR MicroBatchExecution: Query [id = 736cf695-3fca-4dd5-bfa1-271f4661eafd, runId = fb3a105f-1863-4b3f-9cfd-fefb675d4e0b] terminated with error
org.apache.spark.SparkException: [INTERNAL_ERROR] The Spark SQL phase planning failed with an internal error. You hit a bug in Spark or the Spark plugins you use. Please, report this bug to the corresponding communities or vendors, and provide the full stack trace.
	at org.apache.spark.SparkException$.internalError(SparkException.scala:107)
	at org.apache.spark.sql.execution.QueryExecution$.toInternalError(QueryExecution.scala:536)
	at org.apache.spark.sql.execution.QueryExecution$.withInternalError(QueryExecution.scala:548)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:219)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:900)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(QueryExecution.scala:218)
	at org.apache.spark.sql.execution.QueryExec